## Player Types:

- `zs`: An AI agent initialized with a zero-shot prompt. Just the gave description.
- `spp`: Solo Performance Prompting; an AI agent initialized with the SPP prompt.
- `cot`: Chain-of-Thought; an AI agent initialized with the COT prompt.
- `srep`: Singe-Round-Equilibrium-Player; a player who strictly follows the Single Round Equilibrium Strategy (a specific probability distribution over the available moves)
- `pp`: Pattern Player; Follows a cyclic pattern of moves. Always playes moves from this pattern.
- `ap`: Adaptive Player; finds the most frequent move their opponent plays and counters it.
- `tft`: Tit-for-Tat Player; counters opponent's last played move.

## SC experiments:

- We compare an AI agent in a SC environment vs all other agents. If the opponent is also an AI agent, then the opponent in **NOT** a SC player.
- SC: Each time the AI agent has to play. They generate 5 different answers. We then choose the most frequent result and choose an answer that gave that result. We continue with this history for the rest of the game (as many rounds as it is). Conficts are resolved at random and/or by choosing the first answer that gave the result we picked.

In [1]:
models = [
    {
        "id" : "anthropic.claude-3-5-sonnet-20241022-v2:0",
        "name" : "Claude 3.5 Sonnet v2",
        "thinking" : False,
    },
    {
        "id" : "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        "name" : "Claude 3.7 Sonnet",
        "thinking" : False,
    },
    {
        "id" : "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        "name" : "Claude 3.7 Sonnet (Thinking)",
        "thinking" : True,
    },
    {
        "id" : "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "name" : "Claude Sonnet 4",
        "thinking" : False,
    },
    {
        "id" : "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "name" : "Claude Sonnet 4 (Thinking)",
        "thinking" : True,
    },
    {
        "id" : "us.meta.llama3-3-70b-instruct-v1:0",
        "name" : "Llama 3.3 70B Instruct",
        "thinking" : False,
    },
    {
        "id" : "mistral.mistral-large-2407-v1:0",
        "name" : "Mistral Large (24.07)",
        "thinking" : False,
    },
    {
        "id" : "us.deepseek.r1-v1:0",
        "name" : "DeepSeek-R1",
        "thinking" : False,
    },
]

game_settings_types = ["eq1", "eq1-alt", "ba3", "ba3-alt"]

prompt_types = ["default", "spp", "cot"]

In [2]:
import pandas as pd
import os
import json
from collections import defaultdict

def get_total_points_dataframe(
    log_dir: str,
    model_names: list[str],
    prompt_types: list[str],
    game_type: str,
    game_settings_type: str,
    iteration_cnt: int,
    tot: bool,
) -> pd.DataFrame:
    y_replacements = {
        "default": "zs",
    } if not tot else {
        "default": "sc-zs",
        "spp": "sc-spp",
        "cot": "sc-cot",
    }
    x_replacements = {
        "default": "zs",
    } 

    # {(model, prompt) -> {opponent_type -> total_points}}
    heatmap_data = defaultdict(lambda: defaultdict(float))
    opponent_set = set()

    for model in model_names:
        for prompt in prompt_types:
            for itr in range(iteration_cnt):
                directory = os.path.join(log_dir, f"iteration_{itr}", model, game_type, game_settings_type)

                if not os.path.isdir(directory):
                    continue

                for game_dir in sorted(os.listdir(directory)):
                    info_path = os.path.join(directory, game_dir, 'game.json')
                    if not os.path.isfile(info_path):
                        continue

                    with open(info_path) as f:
                        info = json.load(f)

                    player_types = [info.get(f"player_{i}_player_type") for i in range(2)]
                    if prompt not in player_types:
                        continue

                    model_idx = player_types.index(prompt)
                    if model_idx != 0:
                        continue

                    opponent_type = player_types[1 - model_idx]
                    opponent_set.add(opponent_type)

                    total_points = info.get(f"player_{model_idx}_total_points")
                    if total_points is None:
                        raise ValueError(f"Missing total_points for {info_path}")

                    heatmap_data[(model, prompt)][opponent_type] += total_points

    if not heatmap_data:
        raise ValueError("No data collected — check log paths and model+prompt naming conventions.")

    for key in heatmap_data:
        for opponent in heatmap_data[key]:
            heatmap_data[key][opponent] /= iteration_cnt

    opponent_types_aux = ["default", "spp", "cot", "srep", "pp", "ap", "tft"]
    sorted_opponents = [opp for opp in opponent_types_aux if opp in opponent_set]
    model_prompt_keys = [(model, prompt) for model in model_names for prompt in prompt_types]

    # Apply x label replacements
    x_labels = sorted_opponents.copy()
    for old, new in x_replacements.items():
        x_labels = [label.replace(old, new) for label in x_labels]

    rows = []
    index_tuples = []

    for model, prompt in model_prompt_keys:
        new_prompt = prompt
        for old, new in y_replacements.items():
            new_prompt = new_prompt.replace(old, new)

        index_tuples.append( (model, new_prompt) )

        values = []
        for opp in sorted_opponents:
            val = heatmap_data.get((model, prompt), {}).get(opp, -1000)
            values.append(val)
        rows.append(values)


    index = pd.MultiIndex.from_tuples(index_tuples, names=["model", "prompt"])
    df = pd.DataFrame(rows, index=index, columns=x_labels)
    return df


In [3]:
dfs_nonsc_tp = [
    get_total_points_dataframe(
        log_dir="../logs/logs_3/data",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=5,
        tot=False,
    )
    for game_settings_type in game_settings_types
]

dfs_sc_tp = [
    get_total_points_dataframe(
        log_dir="../logs/logs_3/data_tot",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=2,
        tot=True,
    )
    for game_settings_type in game_settings_types
]


In [4]:
prompt_order = ['zs', 'cot', 'spp', 'sc-zs', 'sc-cot', 'sc-spp']
prompt_order_map = {prompt: i for i, prompt in enumerate(prompt_order)}

dfs_merged_tp = []

for df_nonsc, df_sc in zip(dfs_nonsc_tp, dfs_sc_tp):
    df_nonsc = df_nonsc.copy()
    df_sc = df_sc.copy()

    # Prefix the prompt types in sc
    new_index = []
    for model, prompt in df_sc.index:
        new_index.append((model, prompt))
    df_sc.index = pd.MultiIndex.from_tuples(new_index, names=df_sc.index.names)

    # Concatenate vertically
    merged_df = pd.concat([df_nonsc, df_sc])

    # Reorder by (model, prompt) with custom prompt order
    merged_df = merged_df.reset_index()

    # Add a sort key column
    merged_df['prompt_order'] = merged_df['prompt'].map(prompt_order_map)

    # Sort by model, then prompt order
    merged_df = merged_df.sort_values(['model', 'prompt_order'])

    # Drop helper column
    merged_df = merged_df.drop(columns=['prompt_order'])

    # Restore MultiIndex
    merged_df = merged_df.set_index(['model', 'prompt'])

    dfs_merged_tp.append(merged_df)


In [5]:
for df, game_settings_type in zip(dfs_merged_tp, game_settings_types):
    df = df.copy()

    # Identify maxima
    row_max_mask = df.eq(df.max(axis=1), axis=0)
    col_max_mask = df.eq(df.max(axis=0), axis=1)

    # Make values strings, bold if max in row or column
    styled_df = df.copy().astype(str)
    for row in df.index:
        for col in df.columns:
            val = df.loc[row, col]
            is_max = row_max_mask.loc[row, col] or col_max_mask.loc[row, col]
            if pd.isna(val):
                formatted = ""
            else:
                formatted = f"\\textbf{{{val:.1f}}}" if is_max else f"{val:.1f}"
            styled_df.loc[row, col] = formatted

    # Add game_settings_type as MultiIndex header
    styled_df.columns = pd.MultiIndex.from_product(
        [[game_settings_type], styled_df.columns],
    )

    # Output LaTeX
    latex_code = styled_df.to_latex(
        index=True,
        multirow=True,
        multicolumn=True,
        multicolumn_format='c',
        escape=False,  # Allow \textbf
        caption=f"Total Points Averaged Over All Iterations ({game_settings_type})",
        label=f"tab:rps_total_points_avg_heatmap_{game_settings_type}",
    )
    print(latex_code)
    print("\n\n")


\begin{table}
\caption{Total Points Averaged Over All Iterations (eq1)}
\label{tab:rps_total_points_avg_heatmap_eq1}
\begin{tabular}{lllllllll}
\toprule
 &  & \multicolumn{7}{c}{eq1} \\
 &  & zs & spp & cot & srep & pp & ap & tft \\
model & prompt &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{Claude 3.5 Sonnet v2} & zs & 6.6 & -1.2 & -1.8 & -0.4 & 6.0 & 11.2 & \textbf{16.6} \\
 & cot & 0.8 & 2.0 & 5.4 & 0.2 & 3.8 & 6.0 & \textbf{13.0} \\
 & spp & 8.0 & 7.2 & 1.6 & -1.4 & 9.4 & 6.8 & \textbf{12.8} \\
 & sc-zs & -12.5 & -0.5 & 0.5 & -0.5 & 1.5 & 9.0 & \textbf{21.5} \\
 & sc-cot & 9.0 & -1.0 & 0.0 & 1.0 & 10.5 & 14.0 & \textbf{16.5} \\
 & sc-spp & -2.5 & -9.0 & 0.0 & \textbf{7.0} & -1.5 & 5.0 & \textbf{12.5} \\
\cline{1-9}
\multirow[t]{6}{*}{Claude 3.7 Sonnet} & zs & 1.4 & -4.6 & -3.0 & -2.0 & \textbf{13.6} & 8.8 & 4.8 \\
 & cot & 9.6 & 2.6 & 0.4 & 1.4 & 19.6 & 8.2 & \textbf{20.2} \\
 & spp & 3.6 & 4.8 & -5.2 & -2.4 & \textbf{19.2} & 14.2 & 19.0 \\
 & sc-zs & -2.0 & 0.0 & -15.5 & -4

In [6]:
import pandas as pd
import os
import json
from collections import defaultdict

def get_efficiency_dataframe(
    log_dir: str,
    model_names: list[str],
    prompt_types: list[str],
    game_type: str,
    game_settings_type: str,
    iteration_cnt: int,
    tot: bool,
) -> pd.DataFrame:
    y_replacements = {
        "default": "zs",
    } if not tot else {
        "default": "sc-zs",
        "spp": "sc-spp",
        "cot": "sc-cot",
    }
    x_replacements = {
        "default": "zs",
    } 

    # {(model, prompt) -> {opponent_type -> total_points}}
    efficiency_data = defaultdict(lambda: defaultdict(float))
    opponent_set = set()

    for model in model_names:
        for prompt in prompt_types:
            for itr in range(iteration_cnt):
                directory = os.path.join(log_dir, f"iteration_{itr}", model, game_type, game_settings_type)

                if not os.path.isdir(directory):
                    continue

                for game_dir in sorted(os.listdir(directory)):
                    info_path = os.path.join(directory, game_dir, 'game.json')
                    if not os.path.isfile(info_path):
                        continue

                    with open(info_path) as f:
                        info = json.load(f)

                    player_types = [info.get(f"player_{i}_player_type") for i in range(2)]
                    if prompt not in player_types:
                        continue

                    model_idx = player_types.index(prompt)
                    if model_idx != 0:
                        continue

                    opponent_type = player_types[1 - model_idx]
                    opponent_set.add(opponent_type)

                    tokens = info.get(f"player_{model_idx}_tokens")
                    total_points = info.get(f"player_{model_idx}_total_points")

                    if tokens is None or total_points is None:
                        print(f"Model {model}, Prompt {prompt}, Iteration {itr}, Game directory {game_dir} - Missing data in {info_path}")
                        raise ValueError(f"Missing tokens or total_points for {info_path}")

                    efficiency_data[(model, prompt)][opponent_type] += total_points / max(tokens) * 1000  # Scale to per 1000 tokens

    if not efficiency_data:
        raise ValueError("No data collected — check log paths and model+prompt naming conventions.")

    for key in efficiency_data:
        for opponent in efficiency_data[key]:
            efficiency_data[key][opponent] /= iteration_cnt

    opponent_types_aux = ["default", "spp", "cot", "srep", "pp", "ap", "tft"]
    sorted_opponents = [opp for opp in opponent_types_aux if opp in opponent_set]
    model_prompt_keys = [(model, prompt) for model in model_names for prompt in prompt_types]

    for key in efficiency_data:
        aux = 0
        for opp in sorted_opponents:
            aux += efficiency_data[key][opp]
        efficiency_data[key]["avg"] = aux / len(sorted_opponents)

    sorted_opponents.append("avg")  # Add average to the end of the list

    # Apply x label replacements
    x_labels = sorted_opponents.copy()
    for old, new in x_replacements.items():
        x_labels = [label.replace(old, new) for label in x_labels]

    rows = []
    index_tuples = []

    for model, prompt in model_prompt_keys:
        new_prompt = prompt
        for old, new in y_replacements.items():
            new_prompt = new_prompt.replace(old, new)

        index_tuples.append( (model, new_prompt) )

        values = []
        for opp in sorted_opponents:
            val = efficiency_data.get((model, prompt), {}).get(opp, -1000)
            values.append(val)
        rows.append(values)


    index = pd.MultiIndex.from_tuples(index_tuples, names=["model", "prompt"])
    df = pd.DataFrame(rows, index=index, columns=x_labels)
    return df


In [7]:
dfs_nonsc_ef = [
    get_efficiency_dataframe(
        log_dir="../logs/logs_3/data",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=5,
        tot=False,
    )
    for game_settings_type in game_settings_types
]

dfs_sc_ef = [
    get_efficiency_dataframe(
        log_dir="../logs/logs_3/data_tot",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=2,
        tot=True,
    )
    for game_settings_type in game_settings_types
]



In [8]:
prompt_order = ['zs', 'cot', 'spp', 'sc-zs', 'sc-cot', 'sc-spp']
prompt_order_map = {prompt: i for i, prompt in enumerate(prompt_order)}

dfs_merged_ef = []

for df_nonsc, df_sc in zip(dfs_nonsc_ef, dfs_sc_ef):
    df_nonsc = df_nonsc.copy()
    df_sc = df_sc.copy()

    # Prefix the prompt types in sc
    new_index = []
    for model, prompt in df_sc.index:
        new_index.append((model, prompt))
    df_sc.index = pd.MultiIndex.from_tuples(new_index, names=df_sc.index.names)

    # Concatenate vertically
    merged_df = pd.concat([df_nonsc, df_sc])

    # Reorder by (model, prompt) with custom prompt order
    merged_df = merged_df.reset_index()

    # Add a sort key column
    merged_df['prompt_order'] = merged_df['prompt'].map(prompt_order_map)

    # Sort by model, then prompt order
    merged_df = merged_df.sort_values(['model', 'prompt_order'])

    # Drop helper column
    merged_df = merged_df.drop(columns=['prompt_order'])

    # Restore MultiIndex
    merged_df = merged_df.set_index(['model', 'prompt'])

    dfs_merged_ef.append(merged_df)


In [9]:
for df, game_settings_type in zip(dfs_merged_ef, game_settings_types):
    df = df.copy()

    # Identify maxima
    row_max_mask = df.eq(df.max(axis=1), axis=0)
    col_max_mask = df.eq(df.max(axis=0), axis=1)

    # Make values strings, bold if max in row or column
    styled_df = df.copy().astype(str)
    for row in df.index:
        for col in df.columns:
            val = df.loc[row, col]
            is_max = row_max_mask.loc[row, col] or col_max_mask.loc[row, col]
            if pd.isna(val):
                formatted = ""
            else:
                formatted = f"\\textbf{{{val:.2f}}}" if is_max else f"{val:.2f}"
            styled_df.loc[row, col] = formatted

    # Add game_settings_type as MultiIndex header
    styled_df.columns = pd.MultiIndex.from_product(
        [[game_settings_type], styled_df.columns],
    )

    # Output LaTeX
    latex_code = styled_df.to_latex(
        index=True,
        multirow=True,
        multicolumn=True,
        multicolumn_format='c',
        escape=False,  # Allow \textbf
        caption=f"Average Efficiency (Points per kilo-token) ({game_settings_type})",
        label=f"tab:rps_efficiency_avg_heatmap_{game_settings_type}",
    )
    print(latex_code)
    print("\n\n")


\begin{table}
\caption{Average Efficiency (Points per kilo-token) (eq1)}
\label{tab:rps_efficiency_avg_heatmap_eq1}
\begin{tabular}{llllllllll}
\toprule
 &  & \multicolumn{8}{c}{eq1} \\
 &  & zs & spp & cot & srep & pp & ap & tft & avg \\
model & prompt &  &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{Claude 3.5 Sonnet v2} & zs & 1.07 & -0.40 & -0.35 & -0.12 & 0.77 & 2.55 & \textbf{4.13} & 1.09 \\
 & cot & 0.09 & 0.17 & 0.56 & -0.01 & 0.40 & 0.61 & \textbf{1.29} & 0.44 \\
 & spp & 0.66 & 0.57 & 0.11 & -0.13 & 0.79 & 0.50 & \textbf{0.91} & 0.49 \\
 & sc-zs & -0.40 & 0.13 & 0.02 & -0.06 & 0.05 & 0.41 & \textbf{1.22} & 0.20 \\
 & sc-cot & 0.17 & -0.02 & -0.01 & 0.02 & 0.22 & 0.27 & \textbf{0.31} & 0.14 \\
 & sc-spp & -0.08 & -0.16 & -0.01 & 0.10 & -0.03 & 0.08 & \textbf{0.20} & 0.01 \\
\cline{1-10}
\multirow[t]{6}{*}{Claude 3.7 Sonnet} & zs & -0.30 & -0.77 & -0.57 & -0.17 & \textbf{3.38} & 1.66 & 0.84 & 0.58 \\
 & cot & 0.81 & 0.15 & 0.00 & 0.09 & \textbf{1.59} & 0.49 & 1.43 & 0.65 

In [24]:
import pandas as pd
import os
import json
from collections import defaultdict

def get_round_of_understood_opponent(
    log_dir: str,
    model_names: list[str],
    prompt_types: list[str],
    game_type: str,
    game_settings_type: str,
    iteration_cnt: int,
    tot: bool,
) -> pd.DataFrame:
    y_replacements = {
        "default": "zs",
    } if not tot else {
        "default": "sc-zs",
        "spp": "sc-spp",
        "cot": "sc-cot",
    }
    x_replacements = {
        "default": "zs",
    } 

    # {(model, prompt) -> {opponent_type -> total_points}}
    round_data = defaultdict(lambda: defaultdict(float))
    opponent_set = set()

    for model in model_names:
        for prompt in prompt_types:
            for itr in range(iteration_cnt):
                directory = os.path.join(log_dir, f"iteration_{itr}", model, game_type, game_settings_type)

                if not os.path.isdir(directory):
                    continue

                for game_dir in sorted(os.listdir(directory)):
                    info_path = os.path.join(directory, game_dir, 'game.json')
                    if not os.path.isfile(info_path):
                        continue

                    with open(info_path) as f:
                        info = json.load(f)

                    player_types = [info.get(f"player_{i}_player_type") for i in range(2)]
                    if prompt not in player_types:
                        continue

                    model_idx = player_types.index(prompt)
                    if model_idx != 0:
                        continue

                    opponent_type = player_types[1 - model_idx]
                    opponent_set.add(opponent_type)

                    tokens = info.get(f"player_{model_idx}_tokens")
                    points = info.get(f"player_{model_idx}_points")

                    if tokens is None or points is None:
                        print(f"Model {model}, Prompt {prompt}, Iteration {itr}, Game directory {game_dir} - Missing data in {info_path}")
                        raise ValueError(f"Missing tokens or points for {info_path}")

                    win_rates = []
                    wins = 0
                    for rounds_aux, point in enumerate(reversed(points)):
                        rounds = rounds_aux + 1  # Rounds are 1-indexed in the game
                        if point >= 0:
                            wins += 1
                        win_rate = wins / rounds
                        win_rates.insert(0, win_rate)  # Insert at the beginning to keep order
                    
                    target_percentage = 0.9
                    round_of_understood = next((i + 1 for i, rate in enumerate(win_rates) if rate >= target_percentage), len(win_rates) + 1)
                    
                    round_data[(model, prompt)][opponent_type] += round_of_understood


    if not round_data:
        raise ValueError("No data collected — check log paths and model+prompt naming conventions.")

    for key in round_data:
        for opponent in round_data[key]:
            round_data[key][opponent] /= iteration_cnt

    opponent_types_aux = ["default", "spp", "cot", "srep", "pp", "ap", "tft"]
    sorted_opponents = [opp for opp in opponent_types_aux if opp in opponent_set]
    model_prompt_keys = [(model, prompt) for model in model_names for prompt in prompt_types]

    # Apply x label replacements
    x_labels = sorted_opponents.copy()
    for old, new in x_replacements.items():
        x_labels = [label.replace(old, new) for label in x_labels]

    rows = []
    index_tuples = []

    for model, prompt in model_prompt_keys:
        new_prompt = prompt
        for old, new in y_replacements.items():
            new_prompt = new_prompt.replace(old, new)

        index_tuples.append( (model, new_prompt) )

        values = []
        for opp in sorted_opponents:
            val = round_data.get((model, prompt), {}).get(opp, -1000)
            values.append(val)
        rows.append(values)


    index = pd.MultiIndex.from_tuples(index_tuples, names=["model", "prompt"])
    df = pd.DataFrame(rows, index=index, columns=x_labels)
    return df


In [25]:
dfs_nonsc_r = [
    get_round_of_understood_opponent(
        log_dir="../logs/logs_3/data",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=5,
        tot=False,
    )
    for game_settings_type in game_settings_types
]

dfs_sc_r = [
    get_round_of_understood_opponent(
        log_dir="../logs/logs_3/data_tot",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iteration_cnt=2,
        tot=True,
    )
    for game_settings_type in game_settings_types
]



In [26]:
prompt_order = ['zs', 'cot', 'spp', 'sc-zs', 'sc-cot', 'sc-spp']
prompt_order_map = {prompt: i for i, prompt in enumerate(prompt_order)}

dfs_merged_r = []

for df_nonsc, df_sc in zip(dfs_nonsc_r, dfs_sc_r):
    df_nonsc = df_nonsc.copy()
    df_sc = df_sc.copy()

    # Prefix the prompt types in sc
    new_index = []
    for model, prompt in df_sc.index:
        new_index.append((model, prompt))
    df_sc.index = pd.MultiIndex.from_tuples(new_index, names=df_sc.index.names)

    # Concatenate vertically
    merged_df = pd.concat([df_nonsc, df_sc])

    # Reorder by (model, prompt) with custom prompt order
    merged_df = merged_df.reset_index()

    # Add a sort key column
    merged_df['prompt_order'] = merged_df['prompt'].map(prompt_order_map)

    # Sort by model, then prompt order
    merged_df = merged_df.sort_values(['model', 'prompt_order'])

    # Drop helper column
    merged_df = merged_df.drop(columns=['prompt_order'])

    # Restore MultiIndex
    merged_df = merged_df.set_index(['model', 'prompt'])

    dfs_merged_r.append(merged_df)


In [28]:
for df, game_settings_type in zip(dfs_merged_r, game_settings_types):
    df = df.copy()

    # Identify maxima
    row_max_mask = df.eq(df.min(axis=1), axis=0)
    col_max_mask = df.eq(df.min(axis=0), axis=1)

    # Make values strings, bold if min in row or column
    styled_df = df.copy().astype(str)
    for row in df.index:
        for col in df.columns:
            val = df.loc[row, col]
            is_max = row_max_mask.loc[row, col] or col_max_mask.loc[row, col]
            if pd.isna(val):
                formatted = ""
            else:
                formatted = f"\\textbf{{{val:.1f}}}" if is_max else f"{val:.1f}"
            styled_df.loc[row, col] = formatted

    # Add game_settings_type as MultiIndex header
    styled_df.columns = pd.MultiIndex.from_product(
        [[game_settings_type], styled_df.columns],
    )

    # Output LaTeX
    latex_code = styled_df.to_latex(
        index=True,
        multirow=True,
        multicolumn=True,
        multicolumn_format='c',
        escape=False,  # Allow \textbf
        caption=f"Round \\# where the Agent understood the opponent's Strategy ({game_settings_type})",
        label=f"tab:rps_round_heatmap_{game_settings_type}",
    )
    print(latex_code)
    print("\n\n")


\begin{table}
\caption{Round \# where the Agent understood the opponent's Strategy (eq1)}
\label{tab:rps_round_heatmap_eq1}
\begin{tabular}{lllllllll}
\toprule
 &  & \multicolumn{7}{c}{eq1} \\
 &  & zs & spp & cot & srep & pp & ap & tft \\
model & prompt &  &  &  &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{Claude 3.5 Sonnet v2} & zs & 10.6 & 21.4 & 19.6 & 21.0 & 14.6 & 10.4 & \textbf{1.0} \\
 & cot & 17.2 & 19.6 & 20.6 & 19.6 & 16.0 & 22.6 & \textbf{5.8} \\
 & spp & 11.2 & 11.8 & 20.4 & 23.0 & 11.4 & 18.4 & \textbf{1.6} \\
 & sc-zs & 25.0 & 14.0 & 21.5 & 13.5 & 12.0 & 19.5 & \textbf{1.0} \\
 & sc-cot & 15.0 & 20.5 & 22.5 & 25.0 & 12.5 & 7.0 & \textbf{1.0} \\
 & sc-spp & 14.5 & 15.0 & 14.5 & 24.0 & 16.0 & 21.0 & \textbf{1.0} \\
\cline{1-9}
\multirow[t]{6}{*}{Claude 3.7 Sonnet} & zs & 21.8 & 22.2 & 20.2 & 23.4 & \textbf{10.0} & 17.6 & 15.0 \\
 & cot & 15.4 & 16.2 & 18.8 & 18.0 & \textbf{1.0} & 15.6 & \textbf{1.0} \\
 & spp & 24.4 & 20.0 & 22.4 & 23.6 & \textbf{1.2} & 12.0 & \textbf{1.2} \

In [15]:
# Choose the opponent you want to extract
target_opponent = "avg"

# Assume you have:
# - dfs_merged: list of merged DataFrames (one per game_settings_type)
# - game_settings_types: list of corresponding names

# Collect slices for the target opponent from each df
slices = []

for df, setting in zip(dfs_merged_ef, game_settings_types):
    df = df.copy()
    if target_opponent not in df.columns:
        raise ValueError(f"Opponent '{target_opponent}' not found in columns of {setting}.")
    
    # Series with (model, prompt) index and opponent points as values
    s = df[target_opponent].rename(setting)
    slices.append(s)

# Merge all into one DataFrame on index
result_df = pd.concat(slices, axis=1)


In [16]:
def print_latex_heatmap(df):
    df = df.copy()

    # Identify maxima
    row_max_mask = df.eq(df.max(axis=1), axis=0)
    col_max_mask = df.eq(df.max(axis=0), axis=1)

    # Make values strings, bold if min in row or column
    styled_df = df.copy().astype(str)
    for row in df.index:
        for col in df.columns:
            val = df.loc[row, col]
            is_max = row_max_mask.loc[row, col] or col_max_mask.loc[row, col]
            if pd.isna(val):
                formatted = ""
            else:
                formatted = f"\\textbf{{{val:.2f}}}" if is_max else f"{val:.2f}"
            styled_df.loc[row, col] = formatted

    # Output LaTeX
    latex_code = styled_df.to_latex(
        index=True,
        multirow=True,
        multicolumn=True,
        multicolumn_format='c',
        escape=False,  # Allow \textbf
        caption="Average Efficiency (Points per kilo-token)",
        label="tab:pd_efficiency_avg_heatmap",
    )
    print(latex_code)
    print("\n\n")

print_latex_heatmap(result_df)

\begin{table}
\caption{Average Efficiency (Points per kilo-token)}
\label{tab:pd_efficiency_avg_heatmap}
\begin{tabular}{llllll}
\toprule
 &  & eq1 & eq1-alt & ba3 & ba3-alt \\
model & prompt &  &  &  &  \\
\midrule
\multirow[t]{6}{*}{Claude 3.5 Sonnet v2} & zs & \textbf{1.09} & 0.09 & 0.47 & 0.25 \\
 & cot & 0.44 & -0.23 & \textbf{0.49} & -0.03 \\
 & spp & 0.49 & 0.01 & \textbf{0.66} & 0.05 \\
 & sc-zs & 0.20 & 0.10 & \textbf{0.46} & -0.03 \\
 & sc-cot & 0.14 & -0.08 & \textbf{0.15} & -0.01 \\
 & sc-spp & 0.01 & -0.03 & \textbf{0.19} & -0.01 \\
\cline{1-6}
\multirow[t]{6}{*}{Claude 3.7 Sonnet} & zs & 0.58 & 0.20 & \textbf{2.02} & 0.07 \\
 & cot & 0.65 & 0.51 & 0.63 & \textbf{0.71} \\
 & spp & 0.51 & 0.41 & 0.43 & \textbf{0.56} \\
 & sc-zs & -0.08 & 0.00 & 0.00 & \textbf{0.32} \\
 & sc-cot & \textbf{0.21} & 0.14 & 0.17 & 0.11 \\
 & sc-spp & 0.14 & 0.09 & \textbf{0.16} & 0.13 \\
\cline{1-6}
\multirow[t]{6}{*}{Claude 3.7 Sonnet (Thinking)} & zs & 1.92 & \textbf{0.60} & \textbf{3.26} & \t

In [19]:
import pandas as pd
import os
import json
from collections import defaultdict

def get_valid_rate(
    log_dir: str,
    model_names: list[str],
    prompt_types: list[str],
    game_type: str,
    game_settings_type: str,
    iter_cnt: int,
    tot: bool,
) -> pd.DataFrame:
    y_replacements = {
        "zs": "zs",
    } if not tot else {
        "zs": "sc-zs",
        "spp": "sc-spp",
        "cot": "sc-cot",
    }
    x_replacements = {
        "zs": "zs",
    } 

    # {(model, prompt) -> {opponent_type -> total_points}}
    valid_data = defaultdict(lambda: defaultdict(float))
    opponent_set = set()

    for model in model_names:
        for prompt in prompt_types:
            for itr in range(iter_cnt):
                directory = os.path.join(log_dir, f"iteration_{itr}", model, game_type, game_settings_type)

                if not os.path.isdir(directory):
                    continue

                for game_dir in sorted(os.listdir(directory)):
                    info_path = os.path.join(directory, game_dir, 'game.json')
                    if not os.path.isfile(info_path):
                        continue

                    with open(info_path) as f:
                        info = json.load(f)

                    player_types = [info.get(f"player_{i}_player_type") for i in range(2)]
                    if prompt not in player_types:
                        continue

                    model_idx = player_types.index(prompt)
                    if model_idx != 0:
                        continue

                    opponent_type = player_types[1 - model_idx]
                    opponent_set.add(opponent_type)

                    valid_outcomes = info.get("valid_outcomes")

                    if valid_outcomes is None:
                        print(f"Model {model}, Prompt {prompt}, Iteration {itr}, Game directory {game_dir} - Missing data in {info_path}")
                        raise ValueError(f"Missing moves for {info_path}")

                    # find percentage of true valid outcomes
                    total_outcomes = len(valid_outcomes)
                    if total_outcomes == 0:
                        print(f"Model {model}, Prompt {prompt}, Iteration {itr}, Game directory {game_dir} - No valid outcomes in {info_path}")
                        continue

                    valid_count = sum(1 for outcome in valid_outcomes if outcome)
                    valid_rate = valid_count / total_outcomes
                    
                    valid_data[(model, prompt)][opponent_type] += valid_rate


    if not valid_data:
        raise ValueError("No data collected — check log paths and model+prompt naming conventions.")

    for key in valid_data:
        for opponent in valid_data[key]:
            valid_data[key][opponent] /= iter_cnt
            valid_data[key][opponent] *= 100  # Convert to percentage

    opponent_types_aux = ["default", "spp", "cot", "srep", "pp", "mf", "tft"]
    sorted_opponents = [opp for opp in opponent_types_aux if opp in opponent_set]
    model_prompt_keys = [(model, prompt) for model in model_names for prompt in prompt_types]

    # Apply x label replacements
    x_labels = sorted_opponents.copy()
    for old, new in x_replacements.items():
        x_labels = [label.replace(old, new) for label in x_labels]

    rows = []
    index_tuples = []

    for model, prompt in model_prompt_keys:
        new_prompt = prompt
        for old, new in y_replacements.items():
            new_prompt = new_prompt.replace(old, new)

        index_tuples.append( (model, new_prompt) )

        values = []
        for opp in sorted_opponents:
            val = valid_data.get((model, prompt), {}).get(opp, -1000)
            values.append(val)
        rows.append(values)


    index = pd.MultiIndex.from_tuples(index_tuples, names=["model", "prompt"])
    df = pd.DataFrame(rows, index=index, columns=x_labels)
    return df


In [20]:
dfs_nonsc_v = [
    get_valid_rate(
        log_dir="../logs/logs_3/data",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iter_cnt=5,
        tot=False,
    )
    for game_settings_type in game_settings_types
]

dfs_sc_v = [
    get_valid_rate(
        log_dir="../logs/logs_3/data_tot",
        model_names=[model["name"] for model in models],
        prompt_types=prompt_types,
        game_type="rps",
        game_settings_type=game_settings_type,
        iter_cnt=2,
        tot=True,
    )
    for game_settings_type in game_settings_types
]



In [21]:
prompt_order = ['zs', 'cot', 'spp', 'sc-zs', 'sc-cot', 'sc-spp']
prompt_order_map = {prompt: i for i, prompt in enumerate(prompt_order)}

dfs_merged_v = []

for df_nonsc, df_sc in zip(dfs_nonsc_v, dfs_sc_v):
    df_nonsc = df_nonsc.copy()
    df_sc = df_sc.copy()

    # Prefix the prompt types in sc
    new_index = []
    for model, prompt in df_sc.index:
        new_index.append((model, prompt))
    df_sc.index = pd.MultiIndex.from_tuples(new_index, names=df_sc.index.names)

    # Concatenate vertically
    merged_df = pd.concat([df_nonsc, df_sc])

    # Reorder by (model, prompt) with custom prompt order
    merged_df = merged_df.reset_index()

    # Add a sort key column
    merged_df['prompt_order'] = merged_df['prompt'].map(prompt_order_map)

    # Sort by model, then prompt order
    merged_df = merged_df.sort_values(['model', 'prompt_order'])

    # Drop helper column
    merged_df = merged_df.drop(columns=['prompt_order'])

    # Restore MultiIndex
    merged_df = merged_df.set_index(['model', 'prompt'])

    dfs_merged_v.append(merged_df)


In [22]:
model_averages = {}

for df, game_setting in zip(dfs_merged_v, game_settings_types):
    # df has index (model, prompt), and columns = opponent types

    # Step 1: average across opponents
    df_mean_opponents = df.mean(axis=1)  # now index is (model, prompt), values = mean vs opponents

    # Step 2: group by model and average across prompt styles
    avg_by_model = df_mean_opponents.groupby(level=0).mean()  # model -> average

    model_averages[game_setting] = avg_by_model

# Combine into a single DataFrame
df_model_avg = pd.DataFrame(model_averages)

# average across game settings
df_model_avg['avg'] = df_model_avg.mean(axis=1)

# I only want the average column
df_model_avg = df_model_avg[['avg']]


In [ ]:
def print_valid_rates(df):
    df = df.copy()

    # Make values strings, bold if min in row or column
    styled_df = df.copy().astype(str)
    for row in df.index:
        for col in df.columns:
            val = df.loc[row, col]
            is_max = False
            if pd.isna(val):
                formatted = ""
            else:
                formatted = f"\\textbf{{{val:.2f}}}" if is_max else f"{val:.2f}"
            styled_df.loc[row, col] = formatted

    # Output LaTeX
    latex_code = styled_df.to_latex(
        index=True,
        multirow=True,
        multicolumn=True,
        multicolumn_format='c',
        escape=False,  # Allow \textbf
        caption="Average Valid Rate (\\% of Valid Outcomes)",
        label="tab:rps_valid_rates",
    )
    print(latex_code)
    print("\n\n")

print_valid_rates(df_model_avg)

\begin{table}
\caption{Average Valid Rate (\% of Valid Outcomes)}
\label{tab:pd_valid_rates}
\begin{tabular}{ll}
\toprule
 & avg \\
model &  \\
\midrule
Claude 3.5 Sonnet v2 & 100.00 \\
Claude 3.7 Sonnet & 100.00 \\
Claude 3.7 Sonnet (Thinking) & 99.99 \\
Claude Sonnet 4 & 99.91 \\
Claude Sonnet 4 (Thinking) & 100.00 \\
DeepSeek-R1 & 99.99 \\
Llama 3.3 70B Instruct & 100.00 \\
Mistral Large (24.07) & 99.30 \\
\bottomrule
\end{tabular}
\end{table}




